In [3]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
import shutil

In [10]:
## 1- Separar as imagens em grupo de Treinamento e grupo de Testes (70/30)
## --> Do diretório imagens subdivide para imagens_treinamento e imagens_teste

## 1.0. Rebalanceamento
def undersample(src_dir, target_dir, target_count):
    # Verifica o número de imagens na pasta de origem
    src_len = len(os.listdir(src_dir))
    
    # Cria o diretório de destino se não existir
    if not os.path.exists(target_dir):
        target_dir = os.path.join("imagens", target_dir)
        os.makedirs(target_dir)
    
    # Lista de imagens na pasta de origem
    images = os.listdir(src_dir)
    
    # Seleciona aleatoriamente as imagens para serem mantidas
    selected_images = random.sample(images, target_count)
    
    # Move as imagens selecionadas para a pasta de destino
    for img in images:
        if img in selected_images:
            img_src_path = os.path.join(src_dir, img)
            img_target_path = os.path.join(target_dir, img)
            shutil.copy(img_src_path, img_target_path)
            print(f'Imagem movida: {img}')
    
    print(f'Undersampling concluído para a pasta {src_dir}.')

# Definir os diretórios das categorias
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego')
dataset_roma_dir = os.path.join(dataset_dir, 'roma')

# Define o número alvo de imagens para cada categoria (o menor número de imagens entre todas as categorias)
target_count = min(len(os.listdir(dataset_morango_dir)), len(os.listdir(dataset_pessego_dir)), len(os.listdir(dataset_roma_dir)))

# Aplicar undersampling a cada categoria
undersample(dataset_morango_dir, "morango_rebalanceado", target_count)
undersample(dataset_pessego_dir, "pessego_rebalanceado", target_count)
undersample(dataset_roma_dir, "roma_rebalanceado", target_count)

# 1.1. Dataset das imagens
dataset_dir = os.path.join(os.getcwd(), 'imagens')
dataset_morango_dir = os.path.join(dataset_dir, 'morango_rebalanceado')
dataset_pessego_dir = os.path.join(dataset_dir, 'pessego_rebalanceado')
dataset_roma_dir = os.path.join(dataset_dir, 'roma_rebalanceado')

dataset_morango_len = len(os.listdir(dataset_morango_dir))
dataset_pessego_len = len(os.listdir(dataset_pessego_dir))
dataset_roma_len = len(os.listdir(dataset_roma_dir))

print(f'Contagem de imagens de morango rebalanceado: {dataset_morango_len}')
print(f'Contagem de imagens de pêssego rebalanceado: {dataset_pessego_len}')
print(f'Contagem de imagens de romã rebalanceado: {dataset_roma_len}')


Imagem movida: ms_1.jpg
Imagem movida: ms_10.jpg
Imagem movida: ms_100.jpg
Imagem movida: ms_101.jpg
Imagem movida: ms_102.jpg
Imagem movida: ms_103.jpg
Imagem movida: ms_104.jpg
Imagem movida: ms_105.jpg
Imagem movida: ms_106.jpg
Imagem movida: ms_107.jpg
Imagem movida: ms_108.jpg
Imagem movida: ms_109.jpg
Imagem movida: ms_11.jpg
Imagem movida: ms_110.jpg
Imagem movida: ms_111.jpg
Imagem movida: ms_112.jpg
Imagem movida: ms_113.jpg
Imagem movida: ms_114.jpg
Imagem movida: ms_115.jpg
Imagem movida: ms_116.jpg
Imagem movida: ms_117.jpg
Imagem movida: ms_118.jpg
Imagem movida: ms_119.jpg
Imagem movida: ms_12.jpg
Imagem movida: ms_120.jpg
Imagem movida: ms_121.jpg
Imagem movida: ms_122.jpg
Imagem movida: ms_123.jpg
Imagem movida: ms_124.jpg
Imagem movida: ms_125.jpg
Imagem movida: ms_126.jpg
Imagem movida: ms_127.jpg
Imagem movida: ms_128.jpg
Imagem movida: ms_129.jpg
Imagem movida: ms_13.jpg
Imagem movida: ms_130.jpg
Imagem movida: ms_131.jpg
Imagem movida: ms_132.jpg
Imagem movida: ms_

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'c:\\Users\\rdalm\\OneDrive\\Área de Trabalho\\esw-pin3-projeto\\modelo_2\\imagens\\morango_rebalanceado'

In [11]:
## 1.2. Definição e separação dos dados de treinamento e dados de teste
projeto_dir = os.getcwd() # Diretório do projeto
dataset_treinamento_dir = os.path.join(projeto_dir, 'imagens_treinamento') # Caminho para imagens_treinamento
dataset_teste_dir = os.path.join(projeto_dir, 'imagens_teste') # Caminho para imagens_teste

proporcao_dataset_treinamento = 0.8 # 80% será utilizado no treinamento
proporcao_dataset_teste = 1 - proporcao_dataset_treinamento # 20% será utilizado no teste
random_seed = 42 # A resposta para a Vida - segundo livro de Aurélien Géron - gerar com random

classes = ['morango', 'pessego', 'roma']

# Função que executará a separação das imagens (utiliza algoritmo de aleatoriedade)
def split_dataset_to_train_and_test(classe):
    dataset_treinamento_classe_dir = os.path.join(dataset_treinamento_dir, classe) # Dir. destino treinamento classe
    dataset_teste_classe_dir = os.path.join(dataset_teste_dir, classe) # Dir. destino teste classe

    dataset_classe_dir = os.path.join(projeto_dir, 'imagens', classe) # Dir. origem imagens classe
    imagens_classe = [os.path.join(dataset_classe_dir, img) for img in os.listdir(dataset_classe_dir)] # Popular lista com imagens

    imagens_treinamento_classe, imagens_teste_classe = train_test_split(imagens_classe, test_size=proporcao_dataset_teste, random_state=random_seed) # Algoritmo que separa aleatoriamente as imagens de treinamento e de teste

    # Criação dos diretórios de treinamento e teste para a classe
    os.makedirs(dataset_treinamento_classe_dir, exist_ok=True)
    os.makedirs(dataset_teste_classe_dir, exist_ok=True)

    # Copia as imagens de treinamento para a pasta de treinamento da classe em questão
    for imagem in imagens_treinamento_classe:
        shutil.copy(imagem, dataset_treinamento_classe_dir) 

    # Copia as imagens de teste para a pasta de teste da classe em questão
    for imagem in imagens_teste_classe:
        shutil.copy(imagem, dataset_teste_classe_dir) 

# Verifica se os diretórios de treinamento e teste já existem
if not os.path.exists(dataset_treinamento_dir) or not os.path.exists(dataset_teste_dir):
    # Cria os diretórios de treinamento e teste
    os.makedirs(dataset_treinamento_dir, exist_ok=True)
    os.makedirs(dataset_teste_dir, exist_ok=True)
    # Para cada classe, executa a função de divisão de dados
    for classe in classes:
        split_dataset_to_train_and_test(classe)

In [12]:
## 1.3. Como ficou dataset de treinamento e de testes:

dataset_treinamento_morango_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'morango')))
dataset_teste_morango_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango')))
dataset_treinamento_pessego_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'pessego')))
dataset_teste_pessego_len = len(os.listdir(os.path.join(dataset_teste_dir, 'morango')))
dataset_treinamento_roma_len = len(os.listdir(os.path.join(dataset_treinamento_dir, 'roma')))
dataset_teste_roma_len = len(os.listdir(os.path.join(dataset_teste_dir, 'roma')))

print(f'Contagem de imagens de morango para treinamento: {dataset_treinamento_morango_len}')
print(f'Contagem de imagens de morango para teste: {dataset_teste_morango_len}')
print(f'Contagem de imagens de pêssego para treinamento: {dataset_treinamento_pessego_len}')
print(f'Contagem de imagens de pêssego para teste: {dataset_teste_pessego_len}')
print(f'Contagem de imagens de romã para treinamento: {dataset_treinamento_roma_len}')
print(f'Contagem de imagens de romã para teste: {dataset_teste_roma_len}')

# desbalanceamento para a classe de romã
# considerar balancear


Contagem de imagens de morango para treinamento: 200
Contagem de imagens de morango para teste: 50
Contagem de imagens de pêssego para treinamento: 200
Contagem de imagens de pêssego para teste: 50
Contagem de imagens de romã para treinamento: 248
Contagem de imagens de romã para teste: 63


In [13]:
## 2. Pré-processamento das imagens
## --> Definir tamanho de entrada das minhas imagens (em px)
## --> Definir qual estratégia de conversão adotar (scaling da imagem / foco no centro da imagem ignorando periferia / recortar imagem até
## no limite do tamanho definido e ignorar o restante)

image_width = 160
image_heigth = 160
image_size = (image_width, image_heigth)

image_color_channel = 3
image_color_channel_size = 255
image_shape = image_size + (image_color_channel,)

batch_size = 32 # Valor que vou puxar do dataset por vez
epoch = 20 # Quantidade de vezes que vou percorrer meu dataset inteiro
learning_rate = 0.0001 # Taxa de aprendizagem

classes = ['morango', 'pessego', 'roma']

data_set_treinamento = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_treinamento_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

data_set_teste = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_teste_dir,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 648 files belonging to 3 classes.
Found 163 files belonging to 3 classes.
